# 乱数を使って保持器を作る

In [1]:
# `maturin develop` command will install the package in the current virtualenv
!cd ../ && maturin develop

🍹 Building a mixed python/rust project
⚠️  Warning: You're building a library without activating pyo3's `extension-module` feature. See https://pyo3.rs/v0.21.2/building_and_distribution.html#linking
🔗 Found pyo3 bindings
🐍 Found CPython 3.12 at /Users/gyakusu/Morphing/.env/bin/python
   Compiling pyo3-build-config v0.21.2
   Compiling pyo3-macros-backend v0.21.23/353: pyo3-build-config              
   Compiling pyo3-ffi v0.21.2========> ] 343/353: pyo3-macros-backend, pyo3...
   Compiling pyo3 v0.21.2
   Compiling pyo3-macros v0.21.2=====> ] 349/353: pyo3-macros-backend         
   Compiling morphing v0.1.0 (/Users/gyakusu/Morphing)                        
    Finished dev [unoptimized + debuginfo] target(s) in 4.67s                 
📦 Built wheel for CPython 3.12 to /var/folders/w_/pg1jg9jx7hgfsps7wys8dmfm0000gn/T/.tmp2RcMUJ/morphing-0.1.0-cp312-cp312-macosx_11_0_arm64.whl
✏️  Setting installed package as editable
🛠 Installed morphing-0.1.0


In [2]:
from morphing.morphing import CageParameter, Brg


In [3]:
import numpy as np
import pandas as pd

In [4]:
np.random.seed(0)
params = np.random.randn(100, 10) / 3
params[params<-1] = -1
params[params>1] = 1
params[:2]

array([[ 0.58801745,  0.13338574,  0.32624599,  0.7469644 ,  0.62251933,
        -0.32575929,  0.31669614, -0.0504524 , -0.03440628,  0.13686617],
       [ 0.04801452,  0.48475784,  0.25367924,  0.04055834,  0.14795441,
         0.11122478,  0.49802636, -0.06838609,  0.1043559 , -0.28469858]])

In [5]:
params.max(), params.min()

(0.919785038007194, -1.0)

In [6]:
r0 = lambda dr: 2.35e-3 + dr
r1 = lambda dr: 2.85e-3 + dr

r0(0.25e-3), r1(-0.25e-3)

(0.0026, 0.0026)

In [7]:
r0s = r0(params[:, 0] * 0.2e-3)
r1s = r1(params[:, 1] * 0.2e-3)
drs = r1s - r0s

drs.min() * 1e3, drs.max() * 1e3, r0s.min() * 1e3, r1s.max() * 1e3


(0.2846060161200868,
 0.7137738940849623,
 2.1778135578020037,
 2.9786354702544657)

In [8]:
h0 = lambda dh: 0.93e-3 + dh
h1 = lambda dh: 2.10e-3 + dh

h0s = h0(params[:, 2] * 0.2e-3)
h1s = h1(params[:, 3] * 0.2e-3)

h0s.max() * 1e3, h0s.min() * 1e3, h1s.max() * 1e3, h1s.min() * 1e3

(1.0835944465122629,
 0.7796290513731874,
 2.2839570076014386,
 1.9517731231850382)

In [9]:
bevel = lambda r: 0.1 * drs * r

bevels = bevel(params[:, 4]+2)

(bevels / drs).max(), (bevels / drs).min()

(0.2898741350854526, 0.11136092540010864)

In [10]:
pocket_r = lambda dr: 0.825e-3 + dr

pocket_rs = pocket_r(params[:, 5] * 1e-6)
pocket_rs.min() * 1e3, pocket_rs.max() * 1e3

(0.8240551484982418, 0.8258041512265145)

In [11]:
neck_x = lambda dx: 1.70e-3 + dx

neck_xs = neck_x(params[:, 6] * 20e-6)
neck_xs.min() * 1e3, neck_xs.max() * 1e3

(1.6842388412190585, 1.7141119401432245)

In [12]:
neck_r = lambda dr: 1.20e-3 + dr

neck_rs = neck_r(params[:, 7] * 20e-6)
neck_rs.min() * 1e3, neck_rs.max() * 1e3


(1.1817354855520694, 1.2150448233153213)

In [13]:
neck_h = lambda dh: 2.45e-3 + dh

neck_hs = neck_h(params[:, 8] * 100e-6)
neck_hs.min() * 1e3, neck_hs.max() * 1e3


(2.37101376982578, 2.5364808195922715)

In [14]:
neck_dh = lambda r: (neck_hs - h1s) * r * 0.1

neck_dhs = neck_dh(params[:, 9]+2)
neck_dhs.min() * 1e3, neck_dhs.max() * 1e3


(0.028458986531231186, 0.12664880375765164)

In [15]:
df = pd.DataFrame({
    'r0': r0s,
    'r1': r1s,
    'h0': h0s,
    'h1': h1s,
    'bevel': bevels,
    'pocket_r': pocket_rs,
    'neck_x': neck_xs,
    'neck_r': neck_rs,
    'neck_h': neck_hs,
    'neck_dh': neck_dhs
})

# df.to_csv('../data/random/random_params.csv', index=False)
df.to_csv('../data/FEM/random/random_params.csv', index=False)


In [16]:
cage = CageParameter(2.345e-3,2.850e-3,0.93e-3,2.10e-3,0.10e-3,0.825e-3,1.70e-3,1.20e-3,2.45e-3,0.152e-3)
brg  = Brg("../data/FEM/FEM2.vtu", "../data/FEM/index2.xml", cage)
brg_std = brg.std()


In [17]:
new_cage = lambda i: CageParameter(r0s[i], r1s[i], h0s[i], h1s[i], bevels[i], pocket_rs[i], neck_xs[i], neck_rs[i], neck_hs[i], neck_dhs[i])
brg.reload_cage(new_cage(10))


In [18]:
hoge = np.array(brg.get_surface_tetra_and_triangle_as_list())
hoge[:4]

array([[  644,   183,   238,   239],
       [39380,  7560,  7717,  7834],
       [19619,  3753,  3754,  3884],
       [43190,  8402,  8494,  8607]])

In [ ]:
# # cargo run --release "data/Tetra_standard.vtu,data/standard_index.xml,data/Tetra_standard_Smoothed.vtu" "2.345e-3,2.850e-3,0.93e-3,2.10e-3,0.10e-3,0.825e-3,1.70e-3,1.20e-3,2.45e-3,0.152e-3" "1000,1000,5,5"

# command_args0 = lambda i: f"\"data/Tetra_7431.vtu,data/index_7431.xml,data/random/Tetra_7431_{i}.vtu\" "
# command_args0 = lambda i: f"\"data/FEM/FEM2.vtu,data/FEM/index2.xml,data/FEM/random/FEM_{i}.vtu\" "
# command_args0(0)


In [ ]:
# command_args1 = lambda i: f'\"{r0s[i]},{r1s[i]},{h0s[i]},{h1s[i]},{bevels[i]},{pocket_rs[i]},{neck_xs[i]},{neck_rs[i]},{neck_hs[i]},{neck_dhs[i]}\" \"1000,1000,30,5\"'
# command_args1(10)

# # '../data/Tetra_linspace10.vtu 0.0024755433798037503 0.002760149395923837 0.0008453010001009512 0.002164626447210534 4.579193726579163e-05 0.0008256478737285498 0.0016972425401282683 0.0011950169679237283 0.002514098067549346 0.0001742818416754026 1000'

In [ ]:
# import subprocess

# command = "cd ../ && cargo run --release "
# result = subprocess.run(command + command_args0(0) + command_args1(0), shell=True)


In [ ]:
output_span = 10

for i in range(len(params)):
    
    brg.reload_cage(new_cage(i))
    

        
    brg.write_vtk_from_base("../data/FEM/FEM2.vtu", f"../data/FEM/random/FEM2_{i}.vtu")
    
    if i % output_span == 0:
        print(f"Done {i}")


In [ ]:
import os

file_name = lambda i: f'../data/random/Tetra_7431_{i}.vtu'

j = 0
for i in range(100):
    is_exist = os.path.exists(file_name(i))
    
    if not is_exist:
        continue
    
    # change file name
    os.rename(file_name(i), file_name(j))
    j = j + 1

### 次は　`VTK2CGNS.ipynb` へ

[VTK2CGNS.ipynb](VTK2CGNS.ipynb)

## 体積が負のメッシュを可視化



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pyvista as pv


In [ ]:
# ファイル名を指定してデータを読み込む
full_mesh = pv.read(file_name(0))
# full_mesh = pv.read('../data/Tetra_smoothed.vtu')
# full_mesh = pv.read('../data/random/Tetra_linspace0.vtu')
# full_mesh = pv.read('../data/Tetra_7431.vtu')
full_points = np.array(full_mesh.points.tolist()) 
full_points.shape


In [ ]:
full_points[:,0].mean(), full_points[:,1].mean(), full_points[:,2].mean()

In [ ]:
def calculate_volume(mesh):
    # Get the cells from the mesh
    cells = mesh.cells.reshape(-1, 5)[:, 1:]
    
    p0 = mesh.points[cells[:, 0]]
    p1 = mesh.points[cells[:, 1]]
    p2 = mesh.points[cells[:, 2]]
    p3 = mesh.points[cells[:, 3]]
    
    # Calculate the volume of the tetrahedron
    v1 = p1 - p0
    v2 = p2 - p0
    v3 = p3 - p0
    
    # Calculate the volume of the tetrahedron
    volumes = np.abs(np.einsum('ij,ij->i', v1, np.cross(v2, v3))) / 6.0

    return volumes

volumes = calculate_volume(full_mesh)



In [ ]:
plt.figure(figsize=(12, 2))
plt.hist(volumes[volumes<1e-13], bins=100)
plt.xlim([-1e-18,1e-13])
plt.show()

volumes.mean(), volumes.min(), volumes.max()

In [ ]:
import os

In [ ]:
file_name = lambda i: f'../data/random/Tetra_standard_linspace{i}.vtu'

j = 0
for i in range(100):
    is_exist = os.path.exists(file_name(i))
    
    if not is_exist:
        continue
    
    # change file name
    os.rename(file_name(i), file_name(j))
    j = j + 1

In [ ]:
for i in range(80):
    mesh = pv.read(file_name(i))
    volumes = calculate_volume(mesh)
    
    print(f"Volume {i}: {(volumes < 0).sum()}")

In [ ]:
minus_volume_tetra

In [ ]:
# minus_volume_index = cells0[minus_volume_tetra].flatten()
# minus_volume_index = np.unique(minus_volume_index)

# full_points = np.array(full_mesh.points.tolist())
# minus_points = full_points[minus_volume_index]
# minus_points.shape


In [ ]:
surface_index = full_mesh.surface_indices()
minus_volume_index = cells0[minus_volume_tetra]

all_surface = []

for index in cells0:
    bool_array = [i in surface_index for i in index]

    if sum(bool_array) == 4:
        all_surface.append(index)

all_surface = np.array(all_surface).flatten()
all_surface = np.unique(all_surface)

all_points = full_points[all_surface]

minus_surface = []

# for index in cells0:
for index in minus_volume_index:
    bool_array = [i in surface_index for i in index]

    if sum(bool_array) == 4:
        minus_surface.append(index)

minus_surface = np.array(minus_surface).flatten()
minus_surface = np.unique(minus_surface)
minus_surface = np.zeros(full_points.shape, dtype=bool) if minus_surface.shape[0]==0 else minus_surface

In [ ]:
minus_surface.shape

In [ ]:
minus_points = full_points[minus_surface]
surface_points = full_points[surface_index]

minus_points.shape, minus_surface.shape, surface_points.shape

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(12, 3))

for i in [0,1]:
    ax[i].plot(surface_points[:,0], surface_points[:,1+i], '.')
    ax[i].plot(all_points[:,0], all_points[:,1+i], '.')
    # ax[i].plot(minus_points[:,0], minus_points[:,1+i], '.')
    ax[i].axis('equal')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 各点の接続情報を持つ配列を作成する
point_connections = [[] for _ in range(len(full_points))]
for i, cell in enumerate(cells0):
    if i in minus_volume_tetra:
        for j in cell:
            point_connections[j].append(i)

# 各点から繋がっているセルの数を数える
connected_cell_counts = np.array([len(connections) for connections in point_connections])

# ヒストグラムを作成する
plt.figure(figsize=(8, 2))
plt.hist(connected_cell_counts) #, bins=range(max(connected_cell_counts)+2))
plt.xlabel('Number of connected cells per point')
plt.ylabel('Frequency')
plt.title('Histogram of connected negative cells per point')
plt.ylim(0, 200)
plt.show()

In [ ]:
connected_cell_counts.shape, connected_cell_counts.max(), cells0.shape

In [ ]:
num_1 = np.arange(len(connected_cell_counts))[connected_cell_counts==1]
num_1

In [ ]:
import matplotlib.pyplot as plt

# 各点の接続情報を持つ配列を作成する
point_connections1 = [[] for _ in range(len(full_points))]
for i, cell in enumerate(cells0):
    for j in cell:
        if j in num_1:
            point_connections1[j].append(i)

# 各点から繋がっているセルの数を数える
connected_cell_counts1 = np.array([len(connections) for connections in point_connections1])

# ヒストグラムを作成する
plt.figure(figsize=(8, 2))
plt.hist(connected_cell_counts1) #, bins=range(max(connected_cell_counts)+2))
plt.xlabel('Number of connected cells per point')
plt.ylabel('Frequency')
plt.title('Histogram of connected cells per point')
plt.ylim(0, 60)
plt.show()

In [ ]:
example_tetra = minus_volume_tetra[0]
example_index = cells0[example_tetra]
example_point = full_points[example_index]

example_tetra, example_index, example_point

In [ ]:
example_area = np.zeros(4)

for i in range(4):
    j = (i + 1) % 4
    k = (i + 2) % 4
    l = (i + 3) % 4

    vector0 = example_point[l] - example_point[j]
    vector1 = example_point[k] - example_point[j]
    example_area[i] = np.linalg.norm(np.cross(vector0, vector1)) / 2

max_area = example_area.argmax()

example_area, max_area

In [ ]:
j = (max_area + 1) % 4
k = (max_area + 2) % 4
l = (max_area + 3) % 4

example_normal = np.cross(example_point[l] - example_point[j], example_point[k] - example_point[j])
example_normal = example_normal / np.linalg.norm(example_normal)

example_distance = np.dot(example_point[max_area] - example_point[j], example_normal)
example_distance

In [ ]:
example_new_point = example_point[max_area] - 2 * example_normal * example_distance
np.dot(example_new_point - example_point[j], example_normal)

In [ ]:
surface_index = full_mesh.surface_indices()
surface_index

In [ ]:
import numpy as np
index0 = np.array([10, 0, 20, 30])
area0  = np.array([1, 2, 3, 4])
surface0 = np.array([10, 30, 40, 50])

# area0の値が大きい順にインデックスをソートする
sorted_indices = np.argsort(area0)[::-1]

# ソートしたインデックスの中で、surface0に含まれない最初のものを探す
for index in sorted_indices:
    if index0[index] not in surface0:
        result = index0[index]
        break

print(result)


## 以下のアルゴリズムはエッジを反転させる結果，STARでエラーになります．実行しないでください．

In [ ]:
# def flip_cells(mesh, cells_to_flip):
#     # Get the cells from the mesh
#     cells = np.array(mesh.cells.tolist()).reshape(-1, 5)

#     # Flip the cells
#     for cell in cells_to_flip:
#         # Get the points of the tetrahedron
#         points = cells[cell]

#         # Reverse the order of the points
#         cells[cell] = points[[0, 1, 3, 2, 4]]

#     # Update the cells of the mesh
#     mesh.cells = cells.reshape(-1, 1)

#     return mesh

# full_mesh = flip_cells(full_mesh, minus_volume_tetra)
# minus_volume0 = calculate_volume(full_mesh) < 0
# minus_volume0.sum()

In [ ]:
# output_span = 10

# for i in range(100):
#     full_mesh = pv.read(f'../data/random/Tetra_linspace{i}.vtu')
#     volumes = calculate_volume(full_mesh)
#     minus_volume_tetra = np.where(np.array(volumes) < 0)[0]

#     if i % output_span == 0:
#         print(f"itaration {i}, minus_volume_tetra: {minus_volume_tetra.shape}")

#     full_mesh = flip_cells(full_mesh, minus_volume_tetra)
#     volumes = calculate_volume(full_mesh)

#     # pv.save_meshio(f'../data/random/Tetra_linspace{i}.vtu', full_mesh, binary=False)
#     pv.save_meshio(f'../data/random/Tetra_linspace{i}.vtu', full_mesh, binary=True)
    